# 3 Nonlinear Equations and Complementarity Problems 

In [ ]:
#%pylab inline
%pylab notebook
# pylab Populating the interactive namespace from numpy and matplotlib
# numpy for numerical computation
# matplotlib for ploting
import numpy as np
from numpy import append, array, diagonal, tril, triu
from numpy.linalg import inv
from scipy.linalg import lu
#from scipy.linalg import solve
from pprint import pprint
from numpy import array, zeros, diag, diagflat, dot

from sympy import *
import sympy as sym
init_printing()

One of the most basic numerical operations encountered in computational economics
is to find the solution of a system of nonlinear equations. Nonlinear equations generally
arise in one of two forms. In the nonlinear *rootfinding problem*, a function f
mapping $R^n$ to $R^n$ is given and one must compute an n-vector $x$, called a *root* of $f$,
that satisfies

$$f(x) = 0$$


In the nonlinear fixed-point problem, a function $g$ from $R^n$ to $R^n$ is given and one
must compute an n-vector x called a fixed-point of $g$, that satisfies

$$x = g(x)$$


The two forms are equivalent. The rootfinding problem may be recast as a fixed-point
problem by letting $g(x) = x - f(x)$; conversely, the fixed-point problem may be recast
as a rootfinding problem by letting $f(x) = x - g(x)$.


In the related complementarity problem, two n-vectors $a$ and $b$, with $a < b$, and
a function f from $R^n$ to $R^n$ are given, and one must compute an n-vector $x \in [a; b]$,
that satisfies


$$x_i > a_i \rightarrow f_i(x) \forall i = 1,...,n$$

$$x_i < b_i \rightarrow f_i(x) \forall i = 1,...,n$$


The rootfinding problem is a special case of complementarity problem in which $a_i =
-\inf$ and $b_i = +\inf$ for all i. However, the complementarity problem is not simply to
find a root that lies within specified bounds. An element $f_i(x)$ may be nonzero at a
solution of the complementarity problem, provided that $x_i$ equals one of the bounds
$a_i$ or $b_i$.




https://github.com/QuantEcon/QuantEcon.lectures.code


## 3.1 Bisection Method

The bisection method is perhaps the simplest and most robust method for computing
the root of a continuous real-valued function defined on a bounded interval of the real line.



The bisection method is an iterative procedure. Each iteration begins with an
interval known to contain or to bracket a root of f, meaning the function has diffierent
signs at the interval endpoints. The interval is bisected into two subintervals of equal
length.

The bisection method's greatest strength is its robustness. In contrast to other
rootfinding methods, the bisection method is guaranteed to compute a root to a
prescribed tolerance in a known number of iterations, provided valid data are input.


https://lectures.quantecon.org/py/scipy.html#roots-and-fixed-points

https://en.wikipedia.org/wiki/Bisection_method

![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Bisection_method.svg/375px-Bisection_method.svg.png)

In [ ]:
def bisect(f, a, b, tol=10e-5):
    """
    Implements the bisection root finding algorithm, assuming that f is a
    real-valued function on [a, b] satisfying f(a) < 0 < f(b).
    """
    lower, upper = a, b

    while upper - lower > tol:
        middle = 0.5 * (upper + lower)
        # === if root is between lower and middle === #
        if f(middle) > 0:  
            lower, upper = lower, middle
        # === if root is between middle and upper  === #
        else:              
            lower, upper = middle, upper

    return 0.5 * (upper + lower)

In fact SciPy provides it’s own bisection function,

In [ ]:
from scipy.optimize import bisect

f = lambda x: np.sin(4 * (x - 0.25)) + x + x**20 - 1
bisect(f, 0, 1)

## 3.2 Function Iteration
Function iteration is a relatively simple technique that may be used to compute a
fixed-point, $x = g(x)$, of a function from $R^n$ to $R^n$. The technique is also applicable
to a rootfinding problem $f(x) = 0$, by recasting it as the equivalent fixed-point
problem $x = x - f(x)$.


https://en.wikipedia.org/wiki/Fixed-point_iteration


![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8b/Sine_fixed_point.svg/375px-Sine_fixed_point.svg.png)

SciPy has a function for finding (scalar) fixed points too

In [ ]:
from scipy.optimize import fixed_point

fixed_point(lambda x: x**0.5, 0.4)  # 10.0 is an initial guess

## 3.3 Newton's Method


In practice, most nonlinear rootfinding problems are solved using *Newton's method*
or one of its variants. Newton's method is based on the principle of *successive lin-
earization*. Successive linearization calls for a hard nonlinear problem to be replaced
with a sequence of simpler linear problems whose solutions converge to the solution
of the nonlinear problem. Newton's method is typically formulated as a rootfinding
technique, but may be used to solve a fixed-point problem $x = g(x)$ by recasting it
as the rootfinding problem $f(x) = x - g(x) = 0$.


https://en.wikipedia.org/wiki/Newton%27s_method

![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/NewtonIteration_Ani.gif/450px-NewtonIteration_Ani.gif)



In SciPy this algorithm is implemented by scipy.optimize.newton

Unlike bisection, the Newton-Raphson method uses local slope information

This is a double-edged sword:

- When the function is well-behaved, the Newton-Raphson method is faster than bisection
- When the function is less well-behaved, the Newton-Raphson might fail


In [ ]:
from scipy.optimize import newton

f = lambda x: x**3 -2

newton(f, 0.2)   # Start the search at initial condition x = 0.2

### Example: Cournot Duopoly



## 3.4 Quasi-Newton Methods

Quasi-Newton methods offer an alternative to Newton's method for solving rootfinding
problems. Quasi-Newton methods are based on the same successive linearization
principle as Newton's method, except that they replace the Jacobian $f'$ with an
estimate that is easier to compute. Quasi-Newton methods are easier to implement
and less likely to fail due to programming errors than Newton's method because the
analyst need not explicitly code the derivative expressions. Quasi-Newton methods,
however, often converge more slowly than Newton's method and additionally require
the analyst to supply an initial estimate of the function's Jacobian.
The secant method is the most widely used univariate quasi-Newton method. The
secant method is identical to the univariate Newton method, except that it replaces
the derivative of f with a finite-difference approximation constructed from the function
values at the two previous iterates:



https://en.wikipedia.org/wiki/Secant_method

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/92/Secant_method.svg/450px-Secant_method.svg.png)





$${\displaystyle x_{n}=x_{n-1}-f(x_{n-1}){\frac {x_{n-1}-x_{n-2}}{f(x_{n-1})-f(x_{n-2})}}={\frac {x_{n-2}f(x_{n-1})-x_{n-1}f(x_{n-2})}{f(x_{n-1})-f(x_{n-2})}}}  $$

## 3.5 Problems With Newton Methods



Several difficulties commonly arise in the application of Newton and quasi-Newton
methods to solving multivariate non-linear equations. The most common cause of
failure of Newton-type methods is coding errors committed by the analyst. The next
most common cause of failure is the specification of a starting point that is not sufficiently
close to a root. And yet another common cause of failure is an ill-conditioned
Jacobian at the root. These problems can often be mitigated by appropriate action,
though they cannot always be eliminated altogether.

## 3.6 Choosing a Solution Method


Numerical analysts have special terms that they use to classify the rates at which
iterative routines converge. Specifically, a sequence of iterates x(k) is said to converge
to $x^{k}$ at a rate of order p if there is constant $C > 0$ such that




$$ ||x^{(k+1)} - x^*|| <=  C ||x^{(k)} - x^*||^p $$


for suÆciently large k. In particular, the rate of convergence is said to be linear if
$C < 1$ and $p = 1$, superlinear if $1 < p < 2$, and quadratic if $p = 2$.



The asymptotic rates of convergence of the nonlinear equation solution methods
discussed earlier are well known. The bisection method converges at a linear rate
with $C = 1/2$.



## 3.7 Complementarity Problems


Many economic models naturally take the form of a complementary problem rather
than a rootfinding or fixed point problem. In the complementarity problem, two n-
vectors a and b, with a < b, and a function f from $R^n$ to $R^n$  are given, and one must
find an n-vector $x \in [a; b]$, that satisfies




$$x_i > a_i \rightarrow f_i(x) \forall i = 1,...,n$$

$$x_i < b_i \rightarrow f_i(x) \forall i = 1,...,n$$


## 3.8 Complementarity Methods



Although the complementarity problem appears quite diffierent from the ordinary
rootfinding problem, it actually can be reformulated as one. In particular, x solves the
complementarity problem $CP(f; a; b)$ if and only if it solves the rootfinding problem


$$ f(x) =min(max(f(x), a - x), b - x) = 0$$
